In [1]:
import polars as pl
import pandas as pd
from tqdm import tqdm
import os
import json
import numpy as np
import math
from lightgbm import LGBMClassifier
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from polimi.utils._inference import _inference
from ebrec.evaluation.metrics_protocols import *
from ebrec.utils._behaviors import sampling_strategy_wu2019
from polimi.utils._polars import reduce_polars_df_memory_size
from polimi.test.level_2_ensemble.build_model_predictions import require_subsampled_set, train_predict_model
from fastauc.fastauc.fast_auc import CppAuc
import os
import logging
from lightgbm import LGBMClassifier, LGBMRanker
from datetime import datetime
import argparse
import pandas as pd
import joblib
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing_extensions import List, Tuple, Dict, Type
import polars as pl
from polimi.utils._tuning_params import get_models_params
import gc
from polars import testing


In [2]:
N_BOOSTERS = 20
params = {
         "n_estimators": 600,
          "max_depth": 5, 
          "num_leaves": 255,
          "subsample_freq": 4,
          "subsample": 0.6004099528013062, 
          "learning_rate": 0.020058747735265076, 
          "colsample_bytree": 0.28816104133228293, 
          "colsample_bynode": 0.9436687124253154,
          "reg_lambda": 0.0009096841984127709, 
          "reg_alpha": 0.00229692020127837, 
          "min_split_gain": 0.06569239337571059, 
          "min_child_weight": 0.0025913515338086167, 
          "min_child_samples": 53, 
          "extra_trees": True, 
          "max_bin": 8,
          "verbosity": -1,
        }
cpp_auc = CppAuc()


In [3]:
dataset_path = '/home/ubuntu/experiments/stacking/sub_pred_icm'
train_ds = pl.read_parquet('/home/ubuntu/experiments/stacking/sub_pred_icm/train_ds.parquet')
val_ds = pl.read_parquet('/home/ubuntu/experiments/stacking/pred_icm/validation_ds.parquet')
with open(os.path.join(dataset_path, 'data_info.json')) as data_info_file:
        data_info = json.load(data_info_file)

In [4]:
print(train_ds)
if 'postcode' in train_ds.columns:
    train_ds = train_ds.with_columns(pl.col('postcode').fill_null(5))
if 'article_type' in train_ds.columns:
    train_ds = train_ds.with_columns(pl.col('article_type').fill_null('article_default'))
if 'impression_time' in train_ds.columns:
    train_ds = train_ds.drop(['impression_time'])

shape: (298_335, 231)
┌────────────┬─────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ impression ┆ article ┆ predictio ┆ predictio ┆ … ┆ roberta_e ┆ w_2_vec_e ┆ emotions_ ┆ constrast │
│ _id        ┆ ---     ┆ n_catboos ┆ n_catboos ┆   ┆ mb_icm_mi ┆ mb_icm_mi ┆ emb_icm_m ┆ ive_emb_i │
│ ---        ┆ i32     ┆ t_ranker  ┆ t_classif ┆   ┆ nus_media ┆ nus_media ┆ inus_medi ┆ cm_minus_ │
│ u32        ┆         ┆ ---       ┆ ier       ┆   ┆ n_a…      ┆ n_a…      ┆ an_…      ┆ med…      │
│            ┆         ┆ f32       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│            ┆         ┆           ┆ f32       ┆   ┆ f32       ┆ f32       ┆ f32       ┆ f32       │
╞════════════╪═════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 157014     ┆ 9776223 ┆ 0.138048  ┆ 0.181434  ┆ … ┆ -0.923473 ┆ 0.003323  ┆ -2.5146e- ┆ -0.001344 │
│            ┆         ┆           ┆           ┆   ┆           ┆     

In [5]:
train_ds_pandas = train_ds.drop(['impression_id','article','user_id']).to_pandas()
train_ds_pandas[data_info['categorical_columns']] = train_ds_pandas[data_info['categorical_columns']].astype('category')
X = train_ds_pandas.drop(columns=['target'])
y = train_ds_pandas['target']

In [6]:
if 'postcode' in val_ds.columns:
        val_ds = val_ds.with_columns(pl.col('postcode').fill_null(5))
if 'article_type' in val_ds.columns:
        val_ds = val_ds.with_columns(pl.col('article_type').fill_null('article_default'))  
if 'impression_time' in val_ds.columns:
        val_ds = val_ds.drop(columns = ['impression_time']) 
        
val_ds = val_ds.to_pandas()
val_ds[data_info['categorical_columns']] = val_ds[data_info['categorical_columns']].astype('category')

X_val = val_ds[X.columns]
evaluation_ds = pl.from_pandas(val_ds[['impression_id','article', 'target']])

print('Baseline')
model = LGBMClassifier(**params)
model.fit(X, y)
evaluation_ds_copy = evaluation_ds
evaluation_ds_copy = evaluation_ds_copy.with_columns(pl.Series(model.predict_proba(X_val)[:,1]).alias('prediction')).group_by('impression_id').agg(pl.col('target'), pl.col('prediction'))
result = np.mean(
        [cpp_auc.roc_auc_score(np.array(y_t).astype(bool), np.array(y_s).astype(np.float32)) 
            for y_t, y_s in zip(evaluation_ds_copy['target'].to_list(), 
                                evaluation_ds_copy['prediction'].to_list())]
    )
print(f'Baseline AUC: {result}')



/tmp/ipykernel_199078/242159418.py:6: DeprecationWarning: named `columns` param is deprecated; use positional `*args` instead.
  val_ds = val_ds.drop(columns = ['impression_time'])


Baseline
Baseline AUC: 0.8223259084374449


In [7]:
weights = [1/train_ds.shape[0]]*train_ds.shape[0]
train_ds = train_ds.with_columns(pl.Series(weights).alias('weight'))
models = []
models_weights = []

In [8]:
def eval_booster(models_list, models_weights, X_val, evaluation_ds):
    n_models = len(models_list)
    remap = {
        0:-1,
        1:1
    }
    demap={
        -1:0,
        1:1
    }
    evaluation_ds_copy = evaluation_ds
    for n_model in range(n_models):
        evaluation_ds_copy = evaluation_ds_copy.with_columns(pl.Series(models_list[n_model].predict_proba(X_val)[:,1]).alias(f'prediction_{n_model}'))\
            .with_columns((pl.col(f'prediction_{n_model}')* models_weights[n_model]).alias(f'prediction_{n_model}'))
    prediction_ds = evaluation_ds_copy.with_columns(pl.sum_horizontal([f'prediction_{n_model}' for n_model in range(n_models)]).alias('prediction'))\
        .drop([f'prediction_{n_model}' for n_model in range(n_models)])
    evaluation_ds_copy = evaluation_ds_copy.with_columns(pl.Series(prediction_ds['prediction']).alias('prediction'))\
        .group_by('impression_id').agg(pl.col('target'), pl.col('prediction'))
    
    result = np.mean(
        [cpp_auc.roc_auc_score(np.array(y_t).astype(bool), np.array(y_s).astype(np.float32)) 
            for y_t, y_s in zip(evaluation_ds_copy['target'].to_list(), 
                                evaluation_ds_copy['prediction'].to_list())]
    )
    
    
    # for n_model in range(n_models):
    #     evaluation_ds_copy = evaluation_ds_copy.with_columns(pl.Series(models_list[n_model].predict_proba(X_val)[:,1]).alias(f'prediction_{n_model}'))\
    #         .with_columns(pl.col(f'prediction_{n_model}').replace(remap))\
    #         .with_columns((pl.col(f'prediction_{n_model}')* models_weights[n_model]).alias(f'prediction_{n_model}'))
    # prediction_ds = evaluation_ds_copy.with_columns(pl.sum_horizontal([f'prediction_{n_model}' for n_model in range(n_models)]).alias('prediction'))\
    #     .drop([f'prediction_{n_model}' for n_model in range(n_models)])
    # evaluation_ds_copy = evaluation_ds_copy.with_columns(pl.Series(prediction_ds['prediction']).sign().alias('prediction'))\
    #     .with_columns(pl.col('prediction').replace(demap))\
    #     .group_by('impression_id').agg(pl.col('target'), pl.col('prediction'))
    
    # result = np.mean(
    #     [cpp_auc.roc_auc_score(np.array(y_t).astype(bool), np.array(y_s).astype(np.float32)) 
    #         for y_t, y_s in zip(evaluation_ds_copy['target'].to_list(), 
    #                             evaluation_ds_copy['prediction'].to_list())]
    # )
    return result

In [9]:
results = []
MAX_PATIENCE = 5
train_ds = train_ds.with_columns(pl.Series(np.arange(train_ds.shape[0])).alias('index'))
patience = MAX_PATIENCE
last_best = 0
for i in range(N_BOOSTERS):
    print('---------------------------------')
    print(f'Fitting model {i}...')
    print(f'Mama patience is {patience}')
    sampled_items = np.random.choice(train_ds['index'].to_list(), size=int(train_ds.shape[0]*0.3), p=train_ds['weight'].to_list())
    if 'prediction' in train_ds.columns:
        train_ds = train_ds.drop('prediction')
    sampled_items = pl.from_numpy(sampled_items, schema = ['index'])
    x_train = sampled_items.join(train_ds, on='index').drop(['index','weight','impression_id','article','user_id']).to_pandas()
    x_train[data_info['categorical_columns']] = x_train[data_info['categorical_columns']].astype('category')
    y_train = x_train['target']
    x_train = x_train[X.columns]
    # x_train = train_ds.filter(pl.col('index').is_in(sampled_items)).drop(['index','weight','target','impression_id','article','user_id'])
    # y_train = train_ds.filter(pl.col('index').is_in(sampled_items))['target']
    model = LGBMClassifier(**params)
    model.fit(x_train, y_train)
    train_ds = train_ds.with_columns(pl.Series(model.predict(X)).alias('prediction'))
    error = train_ds.filter(pl.col('prediction') != pl.col('target')).select('weight').sum().item()
    print(f'error : {error}')
    if error == 0:
        break
    if error < 0.5:
        r_error = ((1-error)/error)
        alpha = math.log(r_error) * 0.5
        pos_corr = math.exp(-alpha)
        neg_corr = math.exp(alpha)
        train_ds = train_ds.with_columns(pl.when(pl.col('prediction') != pl.col('target'))\
            .then(pl.col('weight') * neg_corr)\
                .otherwise(pl.col('weight')* pos_corr).alias('new_weight'))
        train_ds = train_ds.drop('weight').rename({'new_weight' : 'weight'})\
            .with_columns(pl.col('weight')/pl.col('weight').sum())
        models.append(model)
        models_weights.append(alpha)
        evaluation = eval_booster(models, models_weights, X_val, evaluation_ds)
        print(f'AUC val: {evaluation}')
        patience -= 1
        if last_best < evaluation:
            last_best = evaluation
            patience = MAX_PATIENCE
        else :
            if patience == 0:
                print('Mama is angry now, she has no more patience')
                break
        results.append(evaluation)
    else:
        patience -= 1
        if patience == 0:
            print('Mama is angry now, she has no more patience')
            break

---------------------------------
Fitting model 0...
Mama patience is 5
error : 0.2127909899944693
AUC val: 0.8221738249946411
---------------------------------
Fitting model 1...
Mama patience is 5
error : 0.3251853367430857
AUC val: 0.8212849997370197
---------------------------------
Fitting model 2...
Mama patience is 4
error : 0.38827530533655197
AUC val: 0.8215463910131302
---------------------------------
Fitting model 3...
Mama patience is 3
error : 0.4114394459987322
AUC val: 0.821047885807885
---------------------------------
Fitting model 4...
Mama patience is 2
error : 0.4225632443244527
AUC val: 0.8213156432268337
---------------------------------
Fitting model 5...
Mama patience is 1
error : 0.425258592253537
AUC val: 0.8209208656260508
Mama is angry now, she has no more patience


In [10]:
print(results)

[0.8221738249946411, 0.8212849997370197, 0.8215463910131302, 0.821047885807885, 0.8213156432268337]
